In [1]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64
import random

In [17]:
df = pd.read_csv('data/diabetes_prediction_dataset.csv')

df.head(5)

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [18]:
df.pop('diabetes')

columns = df.columns.values
rand = random.randint(0, len(columns))

features = df.values[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'gender': 'Female',
 'age': 80.0,
 'hypertension': 0,
 'heart_disease': 1,
 'smoking_history': 'never',
 'bmi': 25.19,
 'HbA1c_level': 6.6,
 'blood_glucose_level': 140}

In [19]:
def string_feature(value):
    """
    Convert a string value into a TensorFlow Feature of bytes type.

    Args:
        value (str): The string value to be converted.

    Returns:
        tf.train.Feature: The TensorFlow Feature object.
    """
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, "utf-8")]
        ),
    )

def float_feature(value):
    """
    Convert a float value into a TensorFlow Feature of float type.

    Args:
        value (float): The float value to be converted.

    Returns:
        tf.train.Feature: The TensorFlow Feature object.
    """
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )

def int_feature(value):
    """
    Convert an integer value into a TensorFlow Feature of int type.

    Args:
        value (int): The integer value to be converted.

    Returns:
        tf.train.Feature: The TensorFlow Feature object.
    """
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

In [20]:
def prepare_json(inputs: dict):
    """
    Convert a dictionary of inputs into a JSON string that can be used for prediction.

    Args:
        inputs (dict): A dictionary containing the inputs to be converted. The keys are the names of the inputs and the values are the corresponding values.

    Returns:
        str: A JSON string that contains the inputs in the format required for prediction.
    """
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
        
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [21]:
def make_predictions(inputs):
    """
    Make predictions using the specified inputs by sending a POST request to a TensorFlow Serving model endpoint.

    Args:
        inputs (dict): A dictionary containing the inputs for prediction.

    Returns:
        str: The prediction result, either "Normal" or "Heart Disease" based on the model output.
    """
    json_data = prepare_json(inputs)
    
    endpoint = "http://103.190.215.156:8501/v1/models/diabetes-prediction-model:predict"
    response = requests.post(endpoint, data=json_data)

    prediction = response.json()["predictions"][0][0]

    if prediction:
        prediction_value = prediction
        result = "Normal" if prediction_value < 0.5 else "Diabetes"
    else:
        result = "Error: No prediction found for response."

    print(result) 

In [22]:
make_predictions(inputs)

Diabetes
